In [1]:
import librosa

PATH = "./song.mp3"

SAMPLE_RATE = 11025      # Sample rate for audio files (~11 kHz)

def load_song(file_path, sample_rate=SAMPLE_RATE):
    song, sr = librosa.load(file_path, sr=sample_rate, mono=True)
    return song, sr

song, sample_rate = load_song(PATH)
print(f"Song shape: {song.shape} samples")
print(f"Sample rate: {sample_rate} Hz")

Song shape: (3100320,) samples
Sample rate: 11025 Hz
